<a href="https://colab.research.google.com/github/Didier06/IA_licence_pro_chimie/blob/main/yolo_molecules_aromatiques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================================
# 🌸 TP IA & Chimie - Reconnaissance de molécules de parfumerie
# Université Côte d’Azur — Fablab Valrose
# ============================================================

# --- 1️⃣ Installation des dépendances ---
!pip install rdkit ultralytics tqdm

ERROR: Could not find a version that satisfies the requirement rdkit-pypi (from versions: none)
ERROR: No matching distribution found for rdkit-pypi


In [5]:
# --- 2️⃣ Importations ---
from rdkit import Chem
from rdkit.Chem import Draw
import os, random, shutil
from tqdm import tqdm
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [6]:
# --- 3️⃣ Création du dataset ---
molecules = {
    "Vanillin": "COC1=CC(=CC=C1O)C=O",
    "Limonene": "CC1=CCC(CC1)C(=C)C",
    "Geraniol": "CC(C)=CCCC(C)=CCO",
    "Eugenol": "CC=C(CC1=CC=C(O)C=C1)OC",
    "Menthol": "CC(C)C1CCC(C)CC1O",
    "Citral": "CC(C)=CCC=C(C)C=O",
    "Coumarin": "C1=CC=C2C(=O)OC=CC2=C1"
}

os.makedirs("dataset/images", exist_ok=True)
os.makedirs("dataset/labels", exist_ok=True)

print("🧬 Génération des images moléculaires...")
for i, (name, smi) in enumerate(tqdm(molecules.items())):
    mol = Chem.MolFromSmiles(smi)
    for j in range(10):  # 10 variantes par molécule
        Draw.MolToFile(
            mol,
            f"dataset/images/{name}_{j}.png",
            size=(512,512),
            kekulize=random.choice([True, False]),
            wedgeBonds=random.choice([True, False]),
            includeAtomNumbers=random.choice([True, False])
        )
        with open(f"dataset/labels/{name}_{j}.txt", "w") as f:
            f.write(f"{i} 0.5 0.5 0.9 0.9\n")  # boîte centrée 90%
print("✅ Dataset généré.")

🧬 Génération des images moléculaires...


100%|██████████| 7/7 [00:01<00:00,  6.52it/s]

✅ Dataset généré.


In [7]:
# --- 4️⃣ Création des dossiers train / val ---
for split in ["train", "val"]:
    os.makedirs(f"dataset/images/{split}", exist_ok=True)
    os.makedirs(f"dataset/labels/{split}", exist_ok=True)

images = [f for f in os.listdir("dataset/images") if f.endswith(".png")]
random.shuffle(images)
split_idx = int(0.8 * len(images))
train_imgs, val_imgs = images[:split_idx], images[split_idx:]

def move_files(img_list, split):
    for img_name in img_list:
        label_name = img_name.replace(".png", ".txt")
        shutil.move(os.path.join("dataset/images", img_name), f"dataset/images/{split}/{img_name}")
        shutil.move(os.path.join("dataset/labels", label_name), f"dataset/labels/{split}/{label_name}")

move_files(train_imgs, "train")
move_files(val_imgs, "val")
print(f"📦 {len(train_imgs)} images d'entraînement, {len(val_imgs)} images de validation.")


📦 56 images d'entraînement, 14 images de validation.


In [8]:
# --- 5️⃣ Fichier data.yaml ---
%%writefile data.yaml
train: dataset/images/train
val: dataset/images/val

nc: 7
names: ["Vanillin", "Limonene", "Geraniol", "Eugenol", "Menthol", "Citral", "Coumarin"]

Writing data.yaml


In [ ]:
# --- 6️⃣ Entraînement YOLOv8s (Small) ---
model = YOLO("yolov8s.pt")
model.train(
    data="data.yaml",
    epochs=100,
    imgsz=512,
    batch=8,
    name="mol_recognition_advanced",
    augment=True,
    degrees=15,
    scale=0.2,
    flipud=0.2,
    fliplr=0.5,
    shear=5,
    hsv_h=0.05,
    hsv_s=0.4,
    hsv_v=0.4
)

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=15, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05, hsv_s=0.4, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mol_recognition_advanced, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

In [ ]:
# --- 7️⃣ Évaluation du modèle ---
metrics = model.val()
print("📊 Résultats finaux :")
print(metrics)

# --- 8️⃣ Visualisation des courbes d'entraînement ---
display(Image(filename="runs/detect/mol_recognition_advanced/results.png"))
display(Image(filename="runs/detect/mol_recognition_advanced/confusion_matrix.png"))

In [ ]:
# --- 9️⃣ Analyse des performances ---
df = pd.read_csv("runs/detect/mol_recognition_advanced/results.csv")
plt.figure(figsize=(7,4))
plt.plot(df["epoch"], df["metrics/mAP50"], label="mAP@50")
plt.plot(df["epoch"], df["metrics/precision"], label="Précision")
plt.plot(df["epoch"], df["metrics/recall"], label="Rappel")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("Évolution des performances du modèle")
plt.show()

In [4]:
# --- 🔟  Test sur quelques images ---
test_model = YOLO("runs/detect/mol_recognition_advanced/weights/best.pt")
results = test_model.predict(source="dataset/images/val", show=True, save=True)
print("✅ Résultats enregistrés dans runs/predict/")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🧬 Génération des images moléculaires...


100%|██████████| 7/7 [00:01<00:00,  5.94it/s]

✅ Dataset généré.
📦 56 images d'entraînement, 14 images de validation.



UsageError: Line magic function `%%writefile` not found.


Résumé des actions :

| Élément          | Détail                                       |
| ---------------- | -------------------------------------------- |
| 🔧 Modèle        | YOLOv8s (Small, plus précis que Nano)        |
| 🧬 Données       | 7 molécules × 10 variations = 70 images      |
| 🎨 Augmentation  | rotation, zoom, couleur, symétrie            |
| 📦 Dataset       | split automatique train/val                  |
| 📈 Évaluation    | mAP, précision, rappel, confusion matrix     |
| 💡 Visualisation | courbes d’apprentissage et résultats annotés |


In [ ]:
# --- 11️⃣ Export du modèle pour la MaixCam Pro ---
from ultralytics import YOLO

# Charger le modèle final entraîné
model = YOLO("runs/detect/mol_recognition_advanced/weights/best.pt")

# Exporter en format .kmodel (pour NPU Kendryte K210/K230)
model.export(format="kmodel")  # ⚠️ nécessite Ultralytics ≥ 8.1.0



Une fois l’export terminé, tu obtiens un fichier :

runs/detect/mol_recognition_advanced/weights/best.kmodel

Code Python (MaixPy) pour exécuter le modèle sur la MaixCam

Crée un fichier sur la carte SD, par ex. main.py :

# main.py — MaixCam Pro (K230)
from maix import nn, camera, display
import time

# Charger le modèle YOLO exporté
model = nn.load("/sd/olfactive_model.kmodel")

while True:
    img = camera.capture()
    if img is None:
        continue

    # Exécution du modèle (inférence)
    res = model.forward(img)

    # Affichage à l’écran
    display.show(img)
    if res:
        print("Résultats bruts :", res)
        try:
            obj = res[0]  # première détection
            label = obj['class_name']
            conf = obj['confidence']
            print(f"🧪 Molécule détectée : {label} ({conf:.2f})")
        except:
            pass

    time.sleep(0.1)

🖥️ 14️⃣ Affichage HDMI (optionnel)

Si ta MaixCam est reliée à un écran :

tu verras la molécule capturée par la caméra,

avec le nom détecté affiché dans le terminal ou sur l’écran.

🧠 1️⃣ Que signifie mAP ?

mAP = mean Average Precision
→ c’est la moyenne de la précision sur toutes les classes, en tenant compte de la qualité de localisation des objets.

Autrement dit :

“Le modèle détecte-t-il la bonne molécule, au bon endroit, avec la bonne confiance ?”

📦 2️⃣ Les notions de base : Precision, Recall et IoU
Terme	Signification	En détection
Precision (P)	parmi toutes les détections faites, combien sont correctes ?	peu de faux positifs
Recall (R)	parmi toutes les vraies molécules présentes, combien sont trouvées ?	peu d’oublis
IoU (Intersection over Union)	superposition entre la boîte prédite et la boîte réelle	mesure de “visée”

Exemple schématique :


IoU = aire de l’intersection / aire de l’union

si IoU > 0.5 → la détection est considérée correcte (assez bien localisée)

🧮 3️⃣ Qu’est-ce que mAP@50 ?
➤ Définition :

mAP@50 (ou mAP50) = moyenne de la précision pour IoU ≥ 0.5.

Cela signifie :

Si la boîte prédite recouvre au moins 50 % de la vraie boîte, elle est comptée comme “bonne”.

C’est donc une mesure de précision “tolérante”, idéale pour voir si le modèle reconnaît bien les bons objets.

💡 Interprétation :

mAP@50 = 1.00 → détections parfaites.

mAP@50 = 0.80 → 80 % des molécules bien détectées.

mAP@50 = 0.58 (ton cas actuel) → détection correcte mais pas encore stable.

📊 4️⃣ Qu’est-ce que mAP@50–95 ?
➤ Définition :

Moyenne des mAP calculés pour IoU de 0.5 à 0.95, par pas de 0.05.